<a href="https://colab.research.google.com/github/plusmail/PyWorkspace/blob/master/Keras-08-10.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [6]:
from keras.applications import inception_v3
from keras import backend as K
K.set_learning_phase(0)
model = inception_v3.InceptionV3(weights ='imagenet', include_top = False)


/usr/local/lib/python3.7/dist-packages/tensorflow/python/keras/backend.py:434: UserWarning: `tf.keras.backend.set_learning_phase` is deprecated and will be removed after 2020-10-11. To update it, simply pass a True/False value to the `training` argument of the `__call__` method of your layer or model.
  warnings.warn('`tf.keras.backend.set_learning_phase` is deprecated and '


In [7]:
layer_contributions = { 
    'mixed2' : 0.2,
    'mixed3' : 3.,
    'mixed4' : 2.,
    'mixed5' : 1.5,
}

In [8]:
layer_dict = dict([(layer.name, layer) for layer in model.layers])
loss = K.variable(0.)
for layer_name in layer_contributions:
  coeff = layer_contributions[layer_name]
  activation = layer_dict[layer_name].output

  scaling = K.prod(K.cast(K.shape(activation), 'float32'))
  loss += coeff * K.sum(K.squre(activation[:, 2: -2, 2: -2, :])) / scaling

  dream = model.input
  grads = K.gradients(loss, dream)[0]
  grads /= K.maximum(K.mean(K.abs(grads)), 1e-7)

  outputs = [loss, grads]
  fetch_loss_and_grads = K.fuction([dream],outpus)

def eval_loss_and_grads(x):
  outs = fetch_loss_and_grads([x])
  loss_value = outs[0]
  grad_values = outs[1]
  return loss_value, grad_values

def gradient_ascent(x, iterations ,step, max_loss=None):
  for i in range(interations):
    loss_value, grad_values = eval_loss_and_grads(x)

    if max_loss is not Nonw and loss_value > max_loss:
      break
    print('.....', i , '번째손실 : ', loss_value)
    x += step * grad_values
  return x